<a href="https://colab.research.google.com/github/jankovicsandras/plpgsql_bm25/blob/main/plpgsql_bm25_dev_20241103.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# plpgsql_bm25
## BM25 search implemented in PL/pgSQL


## installing PostgreSQL

In [1]:
! sudo apt install gnupg2 wget nano
! sudo sh -c 'echo "deb http://apt.postgresql.org/pub/repos/apt $(lsb_release -cs)-pgdg main" > /etc/apt/sources.list.d/pgdg.list'
! curl -fsSL https://www.postgresql.org/media/keys/ACCC4CF8.asc | sudo gpg --dearmor -o /etc/apt/trusted.gpg.d/postgresql.gpg
! sudo apt update
! sudo apt install postgresql-16 postgresql-contrib-16 postgresql-server-dev-16
! service postgresql start
! sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
wget is already the newest version (1.21.2-2ubuntu1.1).
gnupg2 is already the newest version (2.2.27-3ubuntu2.1).
Suggested packages:
  hunspell
The following NEW packages will be installed:
  nano
0 upgraded, 1 newly installed, 0 to remove and 49 not upgraded.
Need to get 280 kB of archives.
After this operation, 881 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 nano amd64 6.2-1ubuntu0.1 [280 kB]
Fetched 280 kB in 1s (425 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-p

## init psycopg2 helper

In [2]:
! pip install psycopg2

import psycopg2


def msq(t) :
  with psycopg2.connect("dbname=postgres user=root") as conn:
    with conn.cursor() as cur:
      cur.execute(t)
      try :
        res = cur.fetchall()
        for cdesc in cur.description :
          print(cdesc[0],'|',end='')
        print('')
        for r in res:
          for c in r:
            print(c,'|',end='')
          print('')
      except Exception as ex :
        print(ex)


## Test dataset: Wordpress related QA from Huggingface

In [3]:

! wget https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
! wget https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/queries.jsonl
! ls -la

import random, json


# load from jsonl
wcorpus = []
with open('corpus.jsonl') as f:
  wcstr = f.read()
  wcorpus = wcstr.split('\n')
print('len(wcorpus)',len(wcorpus))

# create sampled corpus, items, questions
sampledwcorpus = random.sample( wcorpus, 10 )
items = []
qqs = []
for i in range(0,len(sampledwcorpus)) :
  wjs = json.loads(sampledwcorpus[i])
  #print(i,'---------------',wjs['_id'])
  #print(len(wjs['title']),wjs['title'])
  #print(len(wjs['text']),wjs['text'])
  items.append( { 'doctext': wjs['text'] } )
  qqs.append( [wjs['title'],i] )

# questions and solutions
random.shuffle(qqs)
questions = [ q[0] for q in qqs ]
questionsolutions = [ q[1]+1 for q in qqs ]


################################ TODO: remove ##################################
#items = [ {'doctext':'one two one five'}, {'doctext':'two three two'}, {'doctext':'three three'}, {'doctext':'five four three four'}, {'doctext':'one five two five five'}, ]
#questions = [ 'three one one', 'five two four two', 'one three three', 'four', 'two five three five' ]
################################################################################


# export to CSV for Postgres
csvfilename = 'items.csv'
with open(csvfilename,'w+') as f:
  f.write('id;doctext\n')
  for i in range(0,len(items)) :
    f.write(str((i+1))+';"'+items[i]['doctext'].replace('"','\'')+'"\n')


# Postgres creating items table by importing from CSV
msq('DROP TABLE IF EXISTS items;')
msq('CREATE TABLE items (id SERIAL, doctext TEXT);')
msq('COPY items FROM \'/content/items.csv\' DELIMITER \';\' CSV HEADER;')
#msq('SELECT * from items;')


--2024-11-04 07:15:39--  https://huggingface.co/datasets/mteb/cqadupstack-wordpress/resolve/main/corpus.jsonl
Resolving huggingface.co (huggingface.co)... 3.165.160.12, 3.165.160.61, 3.165.160.59, ...
Connecting to huggingface.co (huggingface.co)|3.165.160.12|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.hf.co/repos/e3/a1/e3a12a6c68820b63bcd7ae09aa898026eba332004f38239070204b3b146060bc/089b11077372513eca8cc16653485aff1f232f8be18d8c6263be4b3b2bda0078?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27corpus.jsonl%3B+filename%3D%22corpus.jsonl%22%3B&Expires=1730963739&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTczMDk2MzczOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmhmLmNvL3JlcG9zL2UzL2ExL2UzYTEyYTZjNjg4MjBiNjNiY2Q3YWUwOWFhODk4MDI2ZWJhMzMyMDA0ZjM4MjM5MDcwMjA0YjNiMTQ2MDYwYmMvMDg5YjExMDc3MzcyNTEzZWNhOGNjMTY2NTM0ODVhZmYxZjIzMmY4YmUxOGQ4YzYyNjNiZTRiM2IyYmRhMDA3OD9yZXNwb25zZS1jb25

## downloading BM25opt.py

In [4]:
! wget https://raw.githubusercontent.com/jankovicsandras/bm25opt/refs/heads/main/BM25opt.py

--2024-11-04 07:15:41--  https://raw.githubusercontent.com/jankovicsandras/bm25opt/refs/heads/main/BM25opt.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6279 (6.1K) [text/plain]
Saving to: ‘BM25opt.py’

BM25opt.py          100%[===================>]   6.13K  --.-KB/s    in 0s      

2024-11-04 07:15:42 (40.3 MB/s) - ‘BM25opt.py’ saved [6279/6279]



## creating plpgsql_bm25.sql

In [5]:

fstr = """


/*

  plpgsql_bm25.sql
  BM25 Okapi search implemented in PL/pgSQL
  version 2024-09-22 by András Jankovics  https://github.com/jankovicsandras  andras@jankovics.net

  Example usage:
  SELECT bm25createindex( tablename, columnname );  /* tablename and columnname are TEXT types */
  SELECT * FROM bm25topk( tablename, columnname, question, k ); /* question is TEXT, k is INTEGER */

  Please note: bm25createindex will (re)create new tables <tablename>_bm25i_docs and <tablename>_bm25i_words,
  document results will come from <tablename>_bm25i_docs, not the original <tablename>. The algorithm can't
  track changes, so if documents change in the original <tablename>, then bm25createindex() must be called again.

  License: The Unlicense / PUBLIC DOMAIN

*/



/* bm25simpletokenize(): split text to words on whitespace, lowercase, remove some punctiation, similar to mytokenize() */
DROP FUNCTION IF EXISTS bm25simpletokenize;
CREATE OR REPLACE FUNCTION bm25simpletokenize(txt TEXT) RETURNS TEXT[]
LANGUAGE plpgsql
AS $$
DECLARE
  w TEXT;
  w2 TEXT;
  words TEXT[];
BEGIN
  FOREACH w IN ARRAY regexp_split_to_array(LOWER(txt), '\s+') LOOP
    w2 = RTRIM( LTRIM( w, '([{<"''' ), '.?!,:;)]}>"''' );
    IF LENGTH(w2) > 0 THEN
      words = array_append( words, w2 );
    END IF;
  END LOOP;
  RETURN words;
END;
$$;


/* count_words_in_array() creates doc->words counts */
DROP FUNCTION IF EXISTS count_words_in_array;
CREATE OR REPLACE FUNCTION count_words_in_array(input_array text[]) RETURNS jsonb
LANGUAGE plpgsql
AS $$
DECLARE
    word_count jsonb := '{}';
    current_word text;
BEGIN
    FOREACH current_word IN ARRAY input_array LOOP
        IF word_count->>current_word IS NULL THEN
            word_count := jsonb_set( word_count, ARRAY[current_word], '1'::jsonb, true );
        ELSE
            word_count := jsonb_set( word_count, ARRAY[current_word], ((word_count->>current_word)::int + 1)::text::jsonb );
        END IF;
    END LOOP;
    RETURN word_count;
END;
$$;


/* get_word_docs_count() */
DROP FUNCTION IF EXISTS get_word_docs_count;
CREATE OR REPLACE FUNCTION get_word_docs_count( wordstname TEXT, wf JSONB ) RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
  mkey TEXT;
BEGIN
  FOR mkey IN SELECT key FROM jsonb_each_text(wf) LOOP
    EXECUTE FORMAT( 'INSERT INTO %s(word, word_docs_count) VALUES (%s, COALESCE((SELECT word_docs_count FROM %s WHERE word = %s) ,1::INTEGER ) ) ON CONFLICT (word) DO UPDATE SET word_docs_count = (%s.word_docs_count + 1)::INTEGER;', wordstname, quote_literal(mkey), wordstname, quote_literal(mkey), wordstname );
  END LOOP;
END;
$$;


/* get_wsmapobj() */
DROP FUNCTION IF EXISTS get_wsmapobj;
CREATE OR REPLACE FUNCTION get_wsmapobj( docstname TEXT, word TEXT, thisidf DOUBLE PRECISION, thisk1 DOUBLE PRECISION ) RETURNS DOUBLE PRECISION[]
LANGUAGE plpgsql
AS $$
DECLARE
  res DOUBLE PRECISION[];
BEGIN
  /* # self.wsmap[word][i] = thiswordidf * ( word_freqs[i] * (self.k1 + 1) / ( word_freqs[i] + self.hds[i] ) ) */
  EXECUTE FORMAT( 'SELECT ARRAY_AGG( %s * COALESCE(word_freqs->>%s,%s)::INTEGER * %s / ( COALESCE(word_freqs->>%s,%s)::INTEGER + ( %s * hds ) ) ORDER BY id) FROM %s;',
    thisidf, quote_literal(word), quote_literal(0), (thisk1+1), quote_literal(word), quote_literal(0), thisk1, docstname ) INTO res;
  RETURN res;
END;
$$;


/* get_wsmapobjl() */
DROP FUNCTION IF EXISTS get_wsmapobjl;
CREATE OR REPLACE FUNCTION get_wsmapobjl( docstname TEXT, word TEXT, thisidf DOUBLE PRECISION, thisk1 DOUBLE PRECISION, thisdelta DOUBLE PRECISION ) RETURNS DOUBLE PRECISION[]
LANGUAGE plpgsql
AS $$
DECLARE
  res DOUBLE PRECISION[];
BEGIN
  /* # self.wsmap[word][i] = self.idf[word] * twf * (self.k1 + 1) * ( twf/self.hds[di] + self.delta) / (self.k1 + twf/self.hds[di] + self.delta) */
  EXECUTE FORMAT( 'SELECT ARRAY_AGG( %s * COALESCE(word_freqs->>%s,%s)::INTEGER * %s * ( COALESCE(word_freqs->>%s,%s)::INTEGER / hds + %s ) / ( %s + COALESCE(word_freqs->>%s,%s)::INTEGER / hds + %s ) ORDER BY id) FROM %s;',
    thisidf, quote_literal(word), quote_literal(0), (thisk1+1), quote_literal(word), quote_literal(0), thisdelta, thisk1, quote_literal(word), quote_literal(0), thisdelta, docstname ) INTO res;
  RETURN res;
END;
$$;


/* get_wsmapobjplus() */
DROP FUNCTION IF EXISTS get_wsmapobjplus;
CREATE OR REPLACE FUNCTION get_wsmapobjplus( docstname TEXT, word TEXT, thisidf DOUBLE PRECISION, thisk1 DOUBLE PRECISION, thisdelta DOUBLE PRECISION ) RETURNS DOUBLE PRECISION[]
LANGUAGE plpgsql
AS $$
DECLARE
  res DOUBLE PRECISION[];
BEGIN
  /* # self.wsmap[word][i] = self.idf[word] * (self.delta + ( twf * (self.k1 + 1) / ( twf + self.k1 * self.hds[di] ) )) */
  EXECUTE FORMAT( 'SELECT ARRAY_AGG( %s * ( %s + ( COALESCE(word_freqs->>%s,%s)::INTEGER * %s / ( COALESCE(word_freqs->>%s,%s)::INTEGER + %s * hds ) ) ) ORDER BY id) FROM %s;',
    thisidf, thisdelta, quote_literal(word), quote_literal(0), (thisk1+1), quote_literal(word), quote_literal(0), thisk1, docstname ) INTO res;
  RETURN res;
END;
$$;


/* bm25createindex() */
DROP FUNCTION IF EXISTS bm25createindex;
CREATE OR REPLACE FUNCTION bm25createindex(tablename TEXT, columnname TEXT, algo TEXT DEFAULT '') RETURNS VOID
LANGUAGE plpgsql
AS $$
DECLARE
  docstname TEXT := tablename || '_' ||  columnname || '_bm25i_docs' || algo;
  wordstname TEXT := tablename || '_' ||  columnname || '_bm25i_words' || algo;
  param_k1 DOUBLE PRECISION := 1.5;
  param_b DOUBLE PRECISION := 0.75;
  param_epsilon DOUBLE PRECISION := 0.25;
  param_delta DOUBLE PRECISION := 1.0;
  corpus_len INTEGER;
  vocab_len INTEGER;
  total_word_count INTEGER;
  avg_doc_len DOUBLE PRECISION;
  idf_sum DOUBLE PRECISION;
  average_idf DOUBLE PRECISION;
  param_eps DOUBLE PRECISION;
BEGIN

  /* create bm25_params_debug table, this is only required for debugging. */
  /*
  DROP TABLE IF EXISTS bm25_params_debug;
  CREATE TABLE bm25_params_debug ( paramname TEXT PRIMARY KEY, value DOUBLE PRECISION );
  INSERT INTO bm25_params_debug(paramname,value) VALUES('param_k1',param_k1);
  INSERT INTO bm25_params_debug(paramname,value) VALUES('param_b',param_b);
  INSERT INTO bm25_params_debug(paramname,value) VALUES('param_epsilon',param_epsilon);
  */

  /* BM25L and BM25Plus parameters */
  IF algo = 'l' THEN
    param_k1 := 1.5;
    param_b := 0.75;
    param_delta := 0.5;
    param_epsilon := 0.25;
  END IF;

  IF algo = 'plus' THEN
    param_k1 := 1.5;
    param_b := 0.75;
    param_delta := 1;
    param_epsilon := 0.25;
  END IF;

  /* create docs table */
  EXECUTE FORMAT( 'DROP TABLE IF EXISTS %s;', docstname );
  EXECUTE FORMAT( 'CREATE TABLE %s (id SERIAL PRIMARY KEY, doc TEXT, tokenized_doc TEXT[]);', docstname );
  EXECUTE FORMAT( 'INSERT INTO %s (doc, tokenized_doc) SELECT %s AS doc, bm25simpletokenize(%s) AS tokenized_doc FROM %s ;', docstname, columnname, columnname, tablename );

  /* add doc_lens */
  EXECUTE FORMAT( 'ALTER TABLE %s ADD COLUMN doc_lens INTEGER;', docstname );
  EXECUTE FORMAT( 'UPDATE %s SET doc_lens=subquery.doc_lens FROM (SELECT tokenized_doc AS td, CARDINALITY(tokenized_doc) AS doc_lens FROM %s) AS subquery WHERE tokenized_doc = subquery.td;', docstname, docstname );

  /* add word_freqs (JSONB word:count object) */
  EXECUTE FORMAT( 'ALTER TABLE %s ADD COLUMN word_freqs JSONB;', docstname );
  EXECUTE FORMAT( 'UPDATE %s SET word_freqs=count_words_in_array(tokenized_doc);', docstname );

  /* total word count */
  EXECUTE FORMAT( 'SELECT SUM(doc_lens) FROM %s;', docstname ) INTO total_word_count;

  /* this debug statement is not required */
  /*INSERT INTO bm25_params_debug(paramname,value) VALUES('total_word_count',total_word_count);*/

  /* create words table */
  EXECUTE FORMAT( 'DROP TABLE IF EXISTS %s;', wordstname );
  EXECUTE FORMAT( 'CREATE TABLE %s ( word TEXT PRIMARY KEY, word_docs_count INTEGER, idf DOUBLE PRECISION );', wordstname );

  /* count docs with each word */
  EXECUTE FORMAT('SELECT get_word_docs_count( %s, word_freqs ) FROM %s;', quote_literal(wordstname), docstname );

  /* self.avg_doc_len = total_word_count / self.corpus_len */
  EXECUTE FORMAT( 'SELECT COUNT(doc_lens) FROM %s WHERE doc_lens > 0;', docstname ) INTO corpus_len;
  avg_doc_len := total_word_count::DOUBLE PRECISION / corpus_len::DOUBLE PRECISION;

  /* these debug statements are not required */
  /*INSERT INTO bm25_params_debug(paramname,value) VALUES('corpus_len',corpus_len);
  INSERT INTO bm25_params_debug(paramname,value) VALUES('avg_doc_len',avg_doc_len);*/

  /*  # precalc "half of divisor" (1 - self.b + self.b * doc_lens / self.avg_doc_len)  */
  EXECUTE FORMAT( 'ALTER TABLE %s ADD COLUMN hds DOUBLE PRECISION;', docstname );
  EXECUTE FORMAT( 'UPDATE %s SET hds = ( 1.0::DOUBLE PRECISION - %s + %s * doc_lens / %s ) ;', docstname,  param_b, param_b, avg_doc_len );

  /* BM25Okapi IDF | idf = math.log(self.corpus_len - freq + 0.5) - math.log(freq + 0.5) ; self.idf[word] = idf ; idf_sum += idf */
  IF algo = '' THEN
    EXECUTE FORMAT( 'UPDATE %s SET idf = LN( %s - word_docs_count::DOUBLE PRECISION + 0.5::DOUBLE PRECISION) - LN( word_docs_count::DOUBLE PRECISION + 0.5::DOUBLE PRECISION);', wordstname, corpus_len::DOUBLE PRECISION );
    EXECUTE FORMAT( 'SELECT SUM(idf) FROM %s;', wordstname ) INTO idf_sum;
    EXECUTE FORMAT( 'SELECT COUNT(word) FROM %s;', wordstname ) INTO vocab_len;
    average_idf = idf_sum / vocab_len::DOUBLE PRECISION;
    param_eps = param_epsilon * average_idf;
    EXECUTE FORMAT( 'UPDATE %s SET idf = %s WHERE idf < 0;', wordstname, param_eps );
  END IF;

  /* BM25L IDF */
  IF algo = 'l' THEN
    EXECUTE FORMAT( 'UPDATE %s SET idf = LN( %s + 1.0::DOUBLE PRECISION ) - LN( word_docs_count::DOUBLE PRECISION + 0.5::DOUBLE PRECISION);', wordstname, corpus_len::DOUBLE PRECISION );
  END IF;

  /* BM25Plus IDF */
  IF algo = 'plus' THEN
    EXECUTE FORMAT( 'UPDATE %s SET idf = LN( %s + 1.0::DOUBLE PRECISION ) - LN( word_docs_count::DOUBLE PRECISION );', wordstname, corpus_len::DOUBLE PRECISION );
  END IF;

  /* these debug statements are not required */
  /*INSERT INTO bm25_params_debug(paramname,value) VALUES('idf_sum',idf_sum);
  INSERT INTO bm25_params_debug(paramname,value) VALUES('vocab_len',vocab_len);
  INSERT INTO bm25_params_debug(paramname,value) VALUES('average_idf',average_idf);
  INSERT INTO bm25_params_debug(paramname,value) VALUES('param_eps',param_eps);*/

  /*  words * documents score map  */
  EXECUTE FORMAT( 'ALTER TABLE %s ADD COLUMN wsmap DOUBLE PRECISION[];', wordstname );
  IF algo = '' THEN
    EXECUTE FORMAT( 'UPDATE %s SET wsmap = get_wsmapobj( %s, word, idf, %s );', wordstname, quote_literal(docstname), param_k1 );
  END IF;

  IF algo = 'l' THEN
    EXECUTE FORMAT( 'UPDATE %s SET wsmap = get_wsmapobjl( %s, word, idf, %s, %s );', wordstname, quote_literal(docstname), param_k1, param_delta );
  END IF;

  IF algo = 'plus' THEN
    EXECUTE FORMAT( 'UPDATE %s SET wsmap = get_wsmapobjplus( %s, word, idf, %s, %s );', wordstname, quote_literal(docstname), param_k1, param_delta );
  END IF;

END;
$$;


/* bm25scorerows() get the documentscores row for each word */
DROP FUNCTION IF EXISTS bm25scorerows;
CREATE OR REPLACE FUNCTION bm25scorerows(tablename TEXT, mquery TEXT) RETURNS SETOF double precision[]
LANGUAGE plpgsql
AS $$
DECLARE
  w TEXT := '';
BEGIN
  FOR w IN SELECT unnest(bm25simpletokenize(mquery))
  LOOP
    RETURN QUERY EXECUTE FORMAT( 'SELECT wsmap FROM %s WHERE word = %s;', tablename, quote_literal(w) );
  END LOOP;
END;
$$;


/* bm25scoressum(): sums the score rows to one array with the document scores ; TODO: instead of xdocstname maybe with temp table, race condition here? */
DROP FUNCTION IF EXISTS bm25scoressum;
CREATE OR REPLACE FUNCTION bm25scoressum(tablename TEXT, tokenizedquery TEXT) RETURNS SETOF double precision[]
LANGUAGE plpgsql
AS $$
DECLARE
  xdocstname TEXT := tablename || '_bm25i_temp';
BEGIN
  EXECUTE FORMAT( 'DROP TABLE IF EXISTS %s;', xdocstname );
  EXECUTE FORMAT( 'CREATE TABLE %s AS SELECT bm25scorerows(%s, %s);', xdocstname, quote_literal(tablename), quote_literal(tokenizedquery) );
  RETURN QUERY EXECUTE FORMAT( 'SELECT ARRAY_AGG(sum ORDER BY ord) FROM (SELECT ord, SUM(int) FROM %s, unnest(bm25scorerows) WITH ORDINALITY u(int, ord) GROUP BY ord);', xdocstname );
END;
$$;


/* bm25scunnest(): unnests the score array */
DROP FUNCTION IF EXISTS bm25scunnest;
CREATE OR REPLACE FUNCTION bm25scunnest(tablename TEXT, tokenizedquery TEXT) RETURNS TABLE(score double precision)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY SELECT unnest(bm25scoressum(tablename,tokenizedquery));
END;
$$;


/* bm25isc(): returns the index and score of the documents; index starts with 1 */
DROP FUNCTION IF EXISTS bm25isc;
CREATE OR REPLACE FUNCTION bm25isc(tablename TEXT, tokenizedquery TEXT) RETURNS TABLE(id BIGINT, score double precision)
LANGUAGE plpgsql
AS $$
BEGIN
  RETURN QUERY SELECT row_number() OVER () AS id, bm25scunnest FROM bm25scunnest(tablename,tokenizedquery) ;
END;
$$;


/* bm25topk(): returns the index, score and document sorted and limited |  TABLE(id INT, id2 BIGINT, score double precision, doc TEXT) */
DROP FUNCTION IF EXISTS bm25topk;
CREATE OR REPLACE FUNCTION bm25topk(tablename TEXT, columnname TEXT, tokenizedquery TEXT, k INT, algo TEXT DEFAULT '') RETURNS TABLE(id INTEGER, score double precision, doc TEXT)
LANGUAGE plpgsql
AS $$
DECLARE
  docstname TEXT := tablename || '_' ||  columnname || '_bm25i_docs' || algo;
  wordstname TEXT := tablename || '_' ||  columnname || '_bm25i_words' || algo;
BEGIN
  RETURN QUERY EXECUTE FORMAT( 'SELECT t1.id, t2.score, t1.%s AS doc FROM (SELECT id, doc AS %s FROM %s) t1 INNER JOIN ( SELECT id, score FROM bm25isc(%s,%s) ) t2 ON ( t1.id = t2.id ) ORDER BY t2.score DESC LIMIT %s;', columnname, columnname, docstname, quote_literal(wordstname), quote_literal(tokenizedquery), k );
END;
$$;


"""

# save as .sql
filename = 'plpgsql_bm25.sql'
with open(filename,'w+') as f:
  f. write(fstr)

# execute to load the functions
! psql postgresql://@/postgres -f plpgsql_bm25.sql


psql:plpgsql_bm25.sql:25: NOTICE:  function bm25simpletokenize() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:46: NOTICE:  function count_words_in_array() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:67: NOTICE:  function get_word_docs_count() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:82: NOTICE:  function get_wsmapobj() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:98: NOTICE:  function get_wsmapobjl() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:114: NOTICE:  function get_wsmapobjplus() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:130: NOTICE:  function bm25createindex() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:257: NOTICE:  function bm25scorerows() does not exist, skipping
DROP FUNCTION
CREATE FUNCTION
psql:plpgsql_bm25.sql:273: NOTICE:  function bm25scores

## comparative testing of plpgsql_bm25.sql, BM25opt and rank_bm25
#### the order of results and the bm25 scores should be equal

In [6]:

# TODO: for word, freq in word_docs_count.items(): should be  for word, doccount in word_docs_count.items():
# https://github.com/jankovicsandras/bm25opt/blob/63a7ca7399572d7ac86b739ef0f58a88ed2b08b1/BM25opt.py#L104

! pip install rank_bm25

from BM25opt import BM25opt, tokenizer_default
from rank_bm25 import BM25Okapi, BM25L, BM25Plus
import json, time, math


# table and file names
tablename = 'items'
columnname = 'doctext'
p_algo = 'l' #  ''  is BM25Okapi,  'l'  is BM25L,  'plus'  is BM25Plus
docstname = tablename + '_' +  columnname + '_bm25i_docs' + p_algo;
wordstname = tablename + '_' +  columnname + '_bm25i_words' + p_algo;
k = 5


# preparing tokenized corpus
corpus = [ item[columnname] for item in items ]
tokenized_corpus = [ tokenizer_default(document) for document in corpus ]

#rank_bm25_index = BM25Okapi( tokenized_corpus )
#bm25opt_index = BM25opt( corpus, algo='okapi' )

rank_bm25_index = BM25L( tokenized_corpus )
bm25opt_index = BM25opt( corpus, algo='l' )

#rank_bm25_index = BM25Plus( tokenized_corpus )
#bm25opt_index = BM25opt( corpus, algo='plus' )


#rankdebug(rank_bm25_index)
# Postgres has already "items" table, tokenization is built-in
msq('SELECT bm25createindex( \''+tablename+'\', \''+columnname+'\', algo=>\''+p_algo+'\' );')
#msq('SELECT * FROM '+docstname+';')
#msq('SELECT * FROM '+wordstname+';')
#msq('SELECT * FROM bm25_params_debug;')

t1 = 0
t2 = 0
t3 = 0

# Running the questions
for qi,q in enumerate(questions) :

  # tokenize and print question
  tokenizedquestion = tokenizer_default(q)
  if qi % 100 == 0 :
    print('\n----Question',qi,':',q,' | Tokenized: ',tokenizedquestion)
    if questionsolutions and qi<len(questionsolutions) :
      print('Solution ID:',questionsolutions[qi])

  # rank_bm25 BM25 search
  dt1 = time.time()
  doc_scores = rank_bm25_index.get_scores( tokenizedquestion )
  t1 += (time.time()-dt1)
  bres = [ [i,s] for i,s in enumerate(doc_scores) ]
  bres.sort(key=lambda x:x[1],reverse=True)
  bres = bres[:k]

  # BM25opt BM25 search
  dt2 = time.time()
  doc_scores2 = bm25opt_index.get_scores( q )
  t2 += (time.time()-dt2)
  bres2 = [ [i,s] for i,s in enumerate(doc_scores2) ]
  bres2.sort(key=lambda x:x[1],reverse=True)
  bres2 = bres2[:k]

  # Postgres BM25 search
  dt3 = time.time()
  msq( 'SELECT * FROM bm25topk( \''+tablename+'\', \''+columnname+'\', \''+q.replace("'","\'\'")+'\', '+str(k)+', algo=>\''+p_algo+'\' );' )
  t3 += (time.time()-dt3)
  #print('')

  # Print rank_bm25, bm25opt results
  #if qi % 100 == 0 :
  for i in range(0,k):
    print( '|rank_bm25|', bres[i][0]+1, '|',  math.floor(bres[i][1]*10e5 )/10e5, '|', corpus[bres[i][0] ] )
    print( '|bm25opt  |', bres2[i][0]+1, '|', math.floor(bres2[i][1]*10e5)/10e5, '|', corpus[bres2[i][0]] )

# print total times
print('rank_bm25 time:',t1,' | BM25opt time:',t2,' | BM25opt is',t1/t2,'times faster. | plpgsql_bm25 time:',t3)

bm25createindex |
 |

----Question 0 : wpautop() when shortcode attributes are on new lines break args array  | Tokenized:  ['wpautop(', 'when', 'shortcode', 'attributes', 'are', 'on', 'new', 'lines', 'break', 'args', 'array']
Solution ID: 6
id |score |doc |
6 |40.84851090933639 |I have a custom shortcode tag with a few attributes, and I would like to be able to display its attributes on new lines - to make it more readable to content editors:               [component         attr1 ='value1'         attr2 ='value of the second one'         attr3 ='another'         attr4 ='value'         ...         attrN ='valueN']      The reason behind this requirement is that a few attributes might be quite verbose in content. Unfortunately, `wpautop()` adds some nasty extra markup that breaks the `$atts` array like this (using php `print_r($atts)`):               Array ( [0] => attr1 [1] => ='value1'     /> [3] => attr2 = [4] => 'value [5] => of [6] => the [7] => second [8] => one'     /> [10] => '